In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install ftfy regex tqdm
!pip install chess cairosvg pillow numpy pandas matplotlib
!pip install git+https://github.com/openai/CLIP.git


Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/openai/CLIP.git to c:\users\admin\appdata\local\temp\pip-req-build-7m4r7o4j
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\admin\AppData\Local\Temp\pip-req-build-7m4r7o4j'


In [1]:
!ls

Chatgpt_Clip.ipynb
checkpoints
checkpoints.zip
dataset_loader.py
dataset_prep.py
download_pgn.py
lichess_games_2013-01.pgn
train_clip.py


In [2]:
!nvidia-smi


Wed Aug 20 14:50:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.88                 Driver Version: 576.88         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5070 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   48C    P5             35W /  300W |    1961MiB /  16303MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
%%writefile download_pgn.py
import requests
import zstandard as zstd
import os
from tqdm import tqdm
import logging

# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def download_lichess_pgn(year: int, month: int, save_path: str = "."):
    """
    Downloads and decompresses a monthly PGN file from the Lichess Open Database.
    This version handles the modern .zst compression format.

    Args:
        year (int): The year of the games to download (e.g., 2022).
        month (int): The month of the games to download (1-12).
        save_path (str): The directory where the final PGN file will be saved.
    """
    # Format the month to be two digits (e.g., 7 -> 07)
    month_str = f"{month:02d}"

    # Construct the URL for the .zst compressed PGN file
    url = f"https://database.lichess.org/standard/lichess_db_standard_rated_{year}-{month_str}.pgn.zst"

    compressed_file_name = os.path.join(save_path, f"lichess_games_{year}-{month_str}.pgn.zst")
    decompressed_file_name = os.path.join(save_path, f"lichess_games_{year}-{month_str}.pgn")

    logging.info(f"Target URL: {url}")

    try:
        # --- Step 1: Download the compressed file ---
        logging.info(f"Downloading from {url}...")
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes (like 404)

        total_size = int(response.headers.get('content-length', 0))

        with open(compressed_file_name, 'wb') as f, tqdm(
            desc=f"Downloading {os.path.basename(compressed_file_name)}",
            total=total_size,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                bar.update(len(chunk))

        logging.info(f"Successfully downloaded to {compressed_file_name}")

        # --- Step 2: Decompress the .zst file ---
        logging.info(f"Decompressing {compressed_file_name}...")

        dctx = zstd.ZstdDecompressor()
        with open(compressed_file_name, 'rb') as in_file, open(decompressed_file_name, 'wb') as out_file:
            # Use a stream reader for efficient, chunked decompression
            reader = dctx.stream_reader(in_file)
            # Get the file size for the progress bar if possible (requires seeking)
            in_file.seek(0, os.SEEK_END)
            file_size = in_file.tell()
            in_file.seek(0)

            with tqdm(total=file_size, desc=f"Decompressing {os.path.basename(decompressed_file_name)}", unit='iB', unit_scale=True) as pbar:
                while True:
                    chunk = reader.read(16384) # Read in 16KB chunks
                    if not chunk:
                        break
                    out_file.write(chunk)
                    pbar.update(in_file.tell() - pbar.n) # Update progress based on input file read

        logging.info(f"✅ Successfully decompressed to {decompressed_file_name}")

        # --- Step 3: Clean up the compressed file ---
        os.remove(compressed_file_name)
        logging.info(f"Removed temporary file: {compressed_file_name}")

    except requests.exceptions.HTTPError as e:
        logging.error(f"Error: Could not download the file. Status code: {e.response.status_code}")
        logging.error("Please check that the year and month are valid on the Lichess Database page: https://database.lichess.org/")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    # --- Example Usage ---
    # Let's download a recent, smaller file for testing, e.g., from the current year.
    # We'll use January 2024 as an example.
    target_year = 2013
    target_month = 1

    output_directory = "." # Save to the current directory

    # Create the directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    download_lichess_pgn(target_year, target_month, output_directory)

Overwriting download_pgn.py


In [4]:
!pip install zstandard

In [5]:
!python download_pgn.py

2025-08-20 14:50:26,300 - INFO - Target URL: https://database.lichess.org/standard/lichess_db_standard_rated_2013-01.pgn.zst
2025-08-20 14:50:26,300 - INFO - Downloading from https://database.lichess.org/standard/lichess_db_standard_rated_2013-01.pgn.zst...

2025-08-20 14:50:28,636 - INFO - Successfully downloaded to .\lichess_games_2013-01.pgn.zst
2025-08-20 14:50:28,636 - INFO - Decompressing .\lichess_games_2013-01.pgn.zst...

Decompressing lichess_games_2013-01.pgn:   0%|          | 0.00/17.8M [00:00<?, ?iB/s]
Decompressing lichess_games_2013-01.pgn:  86%|########6 | 15.3M/17.8M [00:00<00:00, 153MiB/s]
Decompressing lichess_games_2013-01.pgn: 100%|##########| 17.8M/17.8M [00:00<00:00, 154MiB/s]
2025-08-20 14:50:28,764 - INFO - \u2705 Successfully decompressed to .\lichess_games_2013-01.pgn
2025-08-20 14:50:28,765 - INFO - Removed temporary file: .\lichess_games_2013-01.pgn.zst


In [6]:
%%writefile dataset_prep.py
import chess.pgn
import chess.svg
import cairosvg
from pathlib import Path
from tqdm import tqdm
import logging

# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def pgn_to_examples(pgn_path: str, out_dir: str, max_games: int = 1000):
    """
    Processes a PGN file to generate image-text pairs for training a CLIP model on chess positions.

    Args:
        pgn_path (str): Path to the input PGN file.
        out_dir (str): The root directory to save the generated datasets.
        max_games (int): The maximum number of games to process from the PGN file.
    """
    out_dir = Path(out_dir)
    fen_only_dir = out_dir / "fen_only"
    fen_move_dir = out_dir / "fen_move"

    # Create directories if they don't exist
    fen_only_dir.mkdir(parents=True, exist_ok=True)
    (fen_only_dir / "images").mkdir(exist_ok=True)
    (fen_only_dir / "texts").mkdir(exist_ok=True)

    fen_move_dir.mkdir(parents=True, exist_ok=True)
    (fen_move_dir / "images").mkdir(exist_ok=True)
    (fen_move_dir / "texts").mkdir(exist_ok=True)

    try:
        with open(pgn_path, "r", encoding="utf-8") as f:
            idx = 0
            for _ in tqdm(range(max_games), desc="Processing games"):
                game = chess.pgn.read_game(f)
                if game is None:
                    logging.warning("No more games found in PGN file.")
                    break

                board = game.board()
                for move in game.mainline_moves():
                    fen = board.fen()
                    move_uci = move.uci()

                    # Generate and save the board image
                    svg_data = chess.svg.board(board=board, size=350)
                    try:
                        png_bytes = cairosvg.svg2png(bytestring=svg_data.encode("utf-8"))

                        # Save image for FEN only dataset
                        img_path_fen_only = fen_only_dir / "images" / f"{idx}.png"
                        with open(img_path_fen_only, "wb") as img_file:
                            img_file.write(png_bytes)

                        # Save image for FEN + move dataset
                        img_path_fen_move = fen_move_dir / "images" / f"{idx}.png"
                        with open(img_path_fen_move, "wb") as img_file:
                            img_file.write(png_bytes)

                    except Exception as e:
                        logging.error(f"Failed to convert SVG to PNG for position at index {idx}: {e}")
                        continue

                    # Version A: Save FEN only
                    with open(fen_only_dir / "texts" / f"{idx}.txt", "w", encoding="utf-8") as ftxt:
                        ftxt.write(fen)

                    # Version B: Save FEN + Move
                    with open(fen_move_dir / "texts" / f"{idx}.txt", "w", encoding="utf-8") as ftxt:
                        ftxt.write(f"{fen} | Next move: {move_uci}")

                    board.push(move)
                    idx += 1

            logging.info(f"✅ Successfully created {idx} examples in '{out_dir}'")

    except FileNotFoundError:
        logging.error(f"Error: The file at {pgn_path} was not found.")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

Writing dataset_prep.py


In [7]:
%%writefile dataset_loader.py
import torch
from torch.utils.data import Dataset
from PIL import Image
from pathlib import Path
import clip

class ChessDataset(Dataset):
    """
    A PyTorch Dataset for loading chess board images and their corresponding FEN notations.
    """
    def __init__(self, data_dir: str, preprocess):
        """
        Args:
            data_dir (str): Path to the dataset directory which contains 'images' and 'texts' subdirectories.
            preprocess: The image preprocessing function from CLIP.
        """
        self.data_dir = Path(data_dir)
        self.image_paths = sorted(list((self.data_dir / "images").glob("*.png")))
        self.text_paths = sorted(list((self.data_dir / "texts").glob("*.txt")))
        self.preprocess = preprocess

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx: int):
        # Load image
        image = Image.open(self.image_paths[idx]).convert("RGB")
        image = self.preprocess(image)

        # Load text
        with open(self.text_paths[idx], "r", encoding="utf-8") as f:
            text = f.read().strip()

        # Tokenize text
        tokenized_text = clip.tokenize([text])[0]

        return image, tokenized_text

Writing dataset_loader.py


In [8]:
%%writefile train_clip.py
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
import clip
from dataset_loader import ChessDataset
import argparse
from pathlib import Path
import logging

# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def train_clip_model(data_dir: str, save_path: str, epochs: int, batch_size: int, lr: float, split_ratio: float):
    """
    Trains a CLIP model on the chess dataset.

    Args:
        data_dir (str): Directory of the dataset.
        save_path (str): Directory to save model checkpoints.
        epochs (int): Number of training epochs.
        batch_size (int): Batch size for training.
        lr (float): Learning rate for the optimizer.
        split_ratio (float): Ratio of training data to total data.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    logging.info(f"Using device: {device}")

    # Ensure save directory exists
    Path(save_path).mkdir(parents=True, exist_ok=True)

    model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

    dataset = ChessDataset(data_dir, preprocess)

    # Splitting the dataset
    train_size = int(len(dataset) * split_ratio)
    val_size = len(dataset) - train_size
    train_set, val_set = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9,0.98), eps=1e-6, weight_decay=0.2)
    loss_img = nn.CrossEntropyLoss()
    loss_txt = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for images, texts in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images = images.to(device)
            texts = texts.to(device)

            logits_per_image, logits_per_text = model(images, texts)
            ground_truth = torch.arange(len(images), dtype=torch.long, device=device)

            loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        logging.info(f"Epoch [{epoch+1}/{epochs}] Training Loss: {avg_train_loss:.4f}")

        # Validation phase
        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for images, texts in tqdm(val_loader, desc="Validating"):
                images = images.to(device)
                texts = texts.to(device)
                logits_per_image, logits_per_text = model(images, texts)
                ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
                loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        logging.info(f"Epoch [{epoch+1}/{epochs}] Validation Loss: {avg_val_loss:.4f}")

        # Save model checkpoint
        torch.save(model.state_dict(), f"{save_path}/clip_chess_epoch_{epoch+1}.pt")

    logging.info("✅ Training complete.")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Train a CLIP model on chess data.")
    parser.add_argument("data_dir", type=str, help="Directory of the dataset.")
    parser.add_argument("save_path", type=str, help="Directory to save model checkpoints.")
    parser.add_argument("--epochs", type=int, default=5, help="Number of training epochs.")
    parser.add_argument("--batch_size", type=int, default=64, help="Batch size for training.")
    parser.add_argument("--lr", type=float, default=5e-6, help="Learning rate.")
    parser.add_argument("--split_ratio", type=float, default=0.9, help="Training/validation split ratio.")
    args = parser.parse_args()

    train_clip_model(args.data_dir, args.save_path, args.epochs, args.batch_size, args.lr, args.split_ratio)

Writing train_clip.py


In [9]:
!python dataset_prep.py lichess_games.pgn ./datasets --max_games 500

Traceback (most recent call last):
  File "D:\vivek_projects\vichar\vichar-clip\Notebooks\dataset_prep.py", line 3, in <module>
    import cairosvg
  File "C:\Users\admin\miniconda3\envs\pytorch_5070ti\lib\site-packages\cairosvg\__init__.py", line 25, in <module>
    from . import surface  # noqa isort:skip
  File "C:\Users\admin\miniconda3\envs\pytorch_5070ti\lib\site-packages\cairosvg\surface.py", line 9, in <module>
    import cairocffi as cairo
  File "C:\Users\admin\miniconda3\envs\pytorch_5070ti\lib\site-packages\cairocffi\__init__.py", line 60, in <module>
    cairo = dlopen(
  File "C:\Users\admin\miniconda3\envs\pytorch_5070ti\lib\site-packages\cairocffi\__init__.py", line 57, in dlopen
    raise OSError(error_message)  # pragma: no cover
OSError: no library called "cairo-2" was found
no library called "cairo" was found
no library called "libcairo-2" was found
cannot load library 'libcairo.so.2': error 0x7e.  Additionally, ctypes.util.find_library() did not manage to locate a 

In [45]:
!python train_clip.py ./datasets/fen_only ./checkpoints/fen_only_model --epochs 5 --batch_size 32

2025-08-18 11:57:59,377 - INFO - Using device: cuda
2025-08-18 11:58:04,998 - INFO - Successfully found 30166 examples in the dataset.
Epoch 1/5 Training: 100% 849/849 [03:20<00:00,  4.24it/s]
2025-08-18 12:01:25,461 - INFO - Epoch [1/5] Training Loss: 0.3262
Epoch 1/5 Validating: 100% 95/95 [00:17<00:00,  5.34it/s]
2025-08-18 12:01:43,257 - INFO - Epoch [1/5] Validation Loss: 0.0785
Epoch 2/5 Training: 100% 849/849 [03:18<00:00,  4.28it/s]
2025-08-18 12:05:02,848 - INFO - Epoch [2/5] Training Loss: 0.0703
Epoch 2/5 Validating: 100% 95/95 [00:17<00:00,  5.40it/s]
2025-08-18 12:05:20,444 - INFO - Epoch [2/5] Validation Loss: 0.0408
Epoch 3/5 Training: 100% 849/849 [03:14<00:00,  4.37it/s]
2025-08-18 12:08:35,650 - INFO - Epoch [3/5] Training Loss: 0.0441
Epoch 3/5 Validating: 100% 95/95 [00:17<00:00,  5.37it/s]
2025-08-18 12:08:53,327 - INFO - Epoch [3/5] Validation Loss: 0.0336
Epoch 4/5 Training: 100% 849/849 [03:12<00:00,  4.42it/s]
2025-08-18 12:12:06,434 - INFO - Epoch [4/5] Traini

In [10]:
%%writefile dataset_prep.py
import chess.pgn
import chess.svg
import cairosvg
from pathlib import Path
from tqdm import tqdm
import logging
import argparse

# ======================================================================
# Setup more verbose logging to see all levels of messages
# ======================================================================
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def pgn_to_examples(pgn_path: str, out_dir: str, max_games: int):
    """
    Processes a PGN file to generate image-text pairs for training a CLIP model on chess positions.
    """
    logging.info("--- Starting Dataset Preparation ---")

    # --- Step 1: Validate PGN file existence ---
    pgn_file = Path(pgn_path)
    if not pgn_file.exists():
        logging.error(f"FATAL ERROR: The input PGN file was not found at the specified path: '{pgn_path}'")
        logging.error("Please check if the file name and path are correct.")
        return # Exit the function immediately

    logging.info(f"Found PGN file: '{pgn_path}'")

    # --- Step 2: Set up output directories ---
    out_dir = Path(out_dir)
    fen_only_dir = out_dir / "fen_only"
    fen_move_dir = out_dir / "fen_move"

    try:
        logging.info(f"Creating output directory: '{fen_only_dir}'")
        (fen_only_dir / "images").mkdir(parents=True, exist_ok=True)
        (fen_only_dir / "texts").mkdir(parents=True, exist_ok=True)

        logging.info(f"Creating output directory: '{fen_move_dir}'")
        (fen_move_dir / "images").mkdir(parents=True, exist_ok=True)
        (fen_move_dir / "texts").mkdir(parents=True, exist_ok=True)
        logging.info("Successfully created/verified output directories.")
    except PermissionError:
        logging.error(f"FATAL ERROR: Permission denied to create directories in '{out_dir}'.")
        logging.error("Please check your write permissions for this location.")
        return
    except Exception as e:
        logging.error(f"FATAL ERROR: An unexpected error occurred while creating directories: {e}")
        return

    # --- Step 3: Process the PGN file ---
    try:
        with open(pgn_path, "r", encoding="utf-8") as f:
            game_count = 0
            total_examples = 0

            # Use tqdm to show progress over the number of games
            with tqdm(total=max_games, desc="Processing games") as pbar:
                while True:
                    # Check if we have reached the desired number of games
                    if game_count >= max_games:
                        logging.info(f"Reached the max_games limit of {max_games}.")
                        break

                    game = chess.pgn.read_game(f)

                    if game is None:
                        logging.warning("Finished reading all games from the PGN file before reaching max_games limit.")
                        break

                    board = game.board()
                    # Iterate through moves and create examples
                    for move in game.mainline_moves():
                        fen = board.fen()
                        move_uci = move.uci()

                        # Generate and save the board image
                        svg_data = chess.svg.board(board=board, size=350)
                        try:
                            png_bytes = cairosvg.svg2png(bytestring=svg_data.encode("utf-8"))

                            # Save image for FEN only dataset
                            img_path_fen_only = fen_only_dir / "images" / f"{total_examples}.png"
                            with open(img_path_fen_only, "wb") as img_file:
                                img_file.write(png_bytes)

                            # Save image for FEN + move dataset
                            img_path_fen_move = fen_move_dir / "images" / f"{total_examples}.png"
                            with open(img_path_fen_move, "wb") as img_file:
                                img_file.write(png_bytes)

                        except Exception as e:
                            logging.error(f"Failed to convert SVG to PNG for position at index {total_examples}: {e}")
                            board.push(move) # Still push the move to continue correctly
                            continue

                        # Version A: Save FEN only
                        with open(fen_only_dir / "texts" / f"{total_examples}.txt", "w", encoding="utf-8") as ftxt:
                            ftxt.write(fen)

                        # Version B: Save FEN + Move
                        with open(fen_move_dir / "texts" / f"{total_examples}.txt", "w", encoding="utf-8") as ftxt:
                            ftxt.write(f"{fen} | Next move: {move_uci}")

                        board.push(move)
                        total_examples += 1

                    game_count += 1
                    pbar.update(1) # Update progress bar for each game processed

            logging.info(f"--- Dataset Preparation Complete ---")
            logging.info(f"Processed {game_count} games.")
            logging.info(f"✅ Successfully created {total_examples} image-text pair examples in '{out_dir}'")

    except Exception as e:
        logging.error(f"An unexpected error occurred during PGN processing: {e}", exc_info=True)

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Generate image-text datasets from a PGN chess file.")
    parser.add_argument("pgn_path", type=str, help="Path to the input PGN file.")
    parser.add_argument("out_dir", type=str, help="The root directory to save the generated datasets.")
    parser.add_argument("--max_games", type=int, default=1000, help="Maximum number of games to process from the PGN file.")
    args = parser.parse_args()

    pgn_to_examples(args.pgn_path, args.out_dir, args.max_games)

Overwriting dataset_prep.py


In [8]:
!conda activate pytorch_5070ti



In [ ]:
# !conda install -c conda-forge cairosvg cairocffi cairo pango gdk-pixbuf tinycss2 cssselect2

In [9]:
!python dataset_prep.py lichess_games_2013-01.pgn ./datasets --max_games 500

Traceback (most recent call last):
  File "D:\vivek_projects\vichar\vichar-clip\Notebooks\dataset_prep.py", line 3, in <module>
    import cairosvg
  File "C:\Users\admin\miniconda3\envs\pytorch_5070ti\lib\site-packages\cairosvg\__init__.py", line 25, in <module>
    from . import surface  # noqa isort:skip
  File "C:\Users\admin\miniconda3\envs\pytorch_5070ti\lib\site-packages\cairosvg\surface.py", line 9, in <module>
    import cairocffi as cairo
  File "C:\Users\admin\miniconda3\envs\pytorch_5070ti\lib\site-packages\cairocffi\__init__.py", line 60, in <module>
    cairo = dlopen(
  File "C:\Users\admin\miniconda3\envs\pytorch_5070ti\lib\site-packages\cairocffi\__init__.py", line 57, in dlopen
    raise OSError(error_message)  # pragma: no cover
OSError: no library called "cairo-2" was found
no library called "libcairo-2" was found
cannot load library 'C:\Users\admin\miniconda3\envs\pytorch_5070ti\Library\bin\cairo.dll': error 0x7e
cannot load library 'libcairo.so.2': error 0x7e.  Ad

In [46]:
!python train_clip.py ./datasets/fen_move ./checkpoints/fen_move_model --epochs 5 --batch_size 32

2025-08-18 12:17:32,579 - INFO - Using device: cuda
2025-08-18 12:17:37,564 - INFO - Successfully found 30166 examples in the dataset.
Epoch 1/5 Training: 100% 849/849 [03:34<00:00,  3.96it/s]
2025-08-18 12:21:11,739 - INFO - Epoch [1/5] Training Loss: 0.4330
Epoch 1/5 Validating: 100% 95/95 [00:18<00:00,  5.06it/s]
2025-08-18 12:21:30,520 - INFO - Epoch [1/5] Validation Loss: 0.1398
Epoch 2/5 Training: 100% 849/849 [03:26<00:00,  4.11it/s]
2025-08-18 12:24:58,027 - INFO - Epoch [2/5] Training Loss: 0.0761
Epoch 2/5 Validating: 100% 95/95 [00:17<00:00,  5.31it/s]
2025-08-18 12:25:15,906 - INFO - Epoch [2/5] Validation Loss: 0.0542
Epoch 3/5 Training: 100% 849/849 [03:14<00:00,  4.37it/s]
2025-08-18 12:28:30,882 - INFO - Epoch [3/5] Training Loss: 0.0476
Epoch 3/5 Validating: 100% 95/95 [00:18<00:00,  5.08it/s]
2025-08-18 12:28:49,586 - INFO - Epoch [3/5] Validation Loss: 0.0430
Epoch 4/5 Training: 100% 849/849 [03:12<00:00,  4.41it/s]
2025-08-18 12:32:03,014 - INFO - Epoch [4/5] Traini

In [ ]:
import os
from google.colab import files
import logging

# --- Step 1: Zip the checkpoints folder ---

# Define the folder to be zipped and the name of the output zip file
folder_to_zip = './checkpoints'
zip_filename = 'checkpoints.zip'

logging.info(f"Attempting to zip the folder: '{folder_to_zip}'")

# Check if the folder actually exists
if os.path.isdir(folder_to_zip):
    # The '!' allows us to run a shell command directly in Colab.
    # -r means 'recursive' (include all subdirectories)
    # -q means 'quiet' (to keep the output clean)
    get_ipython().system(f"zip -r -q {zip_filename} {folder_to_zip}")

    logging.info(f"Successfully created zip file: '{zip_filename}'")

    # --- Step 2: Download the created zip file ---
    logging.info("Starting download... Please wait for your browser to prompt you to save the file.")

    try:
        files.download(zip_filename)
        logging.info("Download command issued successfully.")
    except Exception as e:
        logging.error(f"An error occurred during download: {e}")

else:
    logging.error(f"Error: The folder '{folder_to_zip}' does not exist in the current directory.")
    logging.error("Please make sure your training script has run and created the checkpoints.")

In [1]:
!ls

Chatgpt_Clip.ipynb
checkpoints
checkpoints.zip
